In [23]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13265457253283293606
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 83128483840
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10020708011227127968
physical_device_desc: "device: 0, name: NVIDIA H100 80GB HBM3, pci bus id: 0000:00:05.0, compute capability: 9.0"
xla_global_id: 416903419
]


In [24]:
# import os
# import subprocess
# # 파이썬 환경 구축 후 라이브러리 버전 맞춰 설치
# packages_to_install = [
#     "transformers==4.30.2",
#     "torch==2.1.0",
#     "soynlp==0.0.493",
#     "requests==2.31.0",
#     "tensorflow==2.10.0",
#     "accelerate==0.20.1",
#     "kss==4.5.4",
#     "matplotlib==3.7.3",
#     "wordcloud==1.9.2",
#     "JPype1==1.4.1",
#     "rhinoMorph==4.0.1.12",
#     "kiwipiepy==0.16.1",
#     "Konlpy==0.6.0",
#     "nltk==3.8.1",
#     "seaborn==0.13.0"
# ]

# for package in packages_to_install:
#     os.system(f"pip install {package}")

# # pip 업그레이드
# os.system("python -m pip install --upgrade pip")

# # 파이썬 라이브러리 버전 전체 확인
# os.system("pip freeze")

# # 파이썬 라이브러리 버전 하나씩 확인
# packages_to_show = [
#     "transformers",
#     "torch",
#     "soynlp",
#     "requests",
#     "tensorflow",
#     "accelerate",
#     "kss",
#     "matplotlib",
#     "wordcloud",
#     "JPype1",
#     "rhinoMorph",
#     "kiwipiepy",
#     "Konlpy",
#     "nltk",
#     "seaborn"
# ]

# for package in packages_to_show:
#     os.system(f"pip show {package}")


# # git에서 라이브러리 복제
# os.system("git clone https://github.com/ZIZUN/korean-malicious-comments-dataset.git")
# os.system("bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)")

# subprocess.call("pip install git+https://github.com/haven-jeon/PyKoSpacing.git", shell=True)
# print("라이브러리 설치가 완료되었습니다.")


In [25]:
import re
import torch
import pandas as pd

device=torch.device('cuda:0'if torch.cuda.is_available()else 'cpu')
print("device =",device)

device = cuda:0


In [26]:
df = pd.read_excel(r"C:\Users\administrator\Desktop\X_filter\AI_Model\dataset\sentence_data(cpp).xlsx")

df.head()

,Sentence,label
0,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
1,세탁이라고 봐도 된다,0
2,은행에 대출 상담 받으러 가보면 직업의 귀천 바로 알려줌,0
3,ㅋㅋㅋㅋㅋ,0
4,ㄹㅇㅋㅋㅋ,0


In [27]:
null_idx = df[df.label.isnull()].index                             # 해당 index에 null 값 확인
df.loc[null_idx, "Sentence"]                                       # null 값이 존재한 인덱스의 content 값 불러오기

# lable은 content의 가장 끝 문장열로 설정
df.loc[null_idx, "label"] = df.loc[null_idx, "Sentence"].apply(lambda x: x[-1])

# content는 "\t" 앞부분까지의 문자열로 설정
df.loc[null_idx, "Sentence"] = df.loc[null_idx, "Sentence"].apply(lambda x: x[-2])

In [6]:
import sys
sys.path.append("../Morphological_Analyzer/")

In [28]:
# from kiwipiepy import Kiwi
from kiwi import *

kiwi_pre_train = df.sample(frac=0.8, random_state=42)                     # train(80%), test(20%) 셋 구분 
kiwi_pre_test = df.drop(kiwi_pre_train.index)                             # 랜덤으로 샘플링(랜덤으로 숫자 배치)

# 데이터셋 갯수 확인
print('중복 제거 전 학습 데이터셋 : {}'.format(len(kiwi_pre_train)))
print('중복 제거 전 테스트 데이터셋 : {}'.format(len(kiwi_pre_test)))

# 중복 데이터 제거
kiwi_pre_train.drop_duplicates(subset=["Sentence"], inplace=True)
kiwi_pre_test.drop_duplicates(subset=["Sentence"], inplace=True)

# 형태소 분석 적용
train_morp = list(kiwi.tokenize(kiwi_pre_train['Sentence'].tolist(), normalize_coda=True))
test_morp = list(kiwi.tokenize(kiwi_pre_test['Sentence'].tolist(), normalize_coda=True))

train_data = [train_morp, kiwi_pre_train['label']]
test_data = [test_morp, kiwi_pre_test['label']]


# 데이터셋 갯수 확인
print('중복 제거 후 학습 데이터셋 : {}'.format(len(train_data[0])))
print('중복 제거 후 테스트 데이터셋 : {}'.format(len(test_data[0])))

중복 제거 전 학습 데이터셋 : 87998
중복 제거 전 테스트 데이터셋 : 22000
중복 제거 후 학습 데이터셋 : 84546
중복 제거 후 테스트 데이터셋 : 21774


In [8]:
# 각 문장에서 최대 토큰 수 계산
max_tokens = max(len(sentence) for sentence in train_data[0])

In [9]:
# DataFrame 생성
train_df = pd.DataFrame(index=range(len(train_data[0])), columns=range(max_tokens + 1))

# 각 문장에 대해 반복하면서 DataFrame 생성
for i, sentence in enumerate(train_data[0]):
    # DataFrame에 토큰 형태 추가
    for j, token in enumerate(sentence):
        train_df.loc[i, j] = token.form

    # 마지막 열에 'label' 값 추가
    train_df.loc[i, max_tokens] = train_data[1].iloc[i]

# NaN 값을 빈 문자열로 대체
train_df = train_df.fillna('')

# 결과적인 DataFrame 출력
print(train_df)

       0    1    2    3     4    5   6       7     8    9    ... 639 640 641  \
0        음   그렇   ᆫ가    ?                                    ...               
1      강원도    는   꼴랑  700     억    주   고  ㅋㅋㅋㅋㅋㅋ   차이나   타운  ...               
2        '    '  빨갱이    '     '    '   들      이제     ᆫ  대놓고  ...               
3       그런   대사    뭐    이     지    ,   ?      소설     같    네  ...               
4        '    '   미치    ᆫ     '    '   '       .  그야말로   사람  ...               
...    ...  ...  ...  ...   ...  ...  ..     ...   ...  ...  ...  ..  ..  ..   
84541   키키   같이   입사   도전     ?                              ...               
84542   애초    에  유재석    은  우리나라  개돼지  국민       서     으    로  ...               
84543  대가리    텅    비    ᆫ     년    들  ..                     ...               
84544    불    에    타    는     줄    알   았       음   ...    덥  ...               
84545  아니요  ...   아무    도     없   네요                         ...               

      642 643 644 645 646 647 648  
0  

In [10]:

# DataFrame 생성
test_df = pd.DataFrame(index=range(len(test_data[0])), columns=range(max_tokens + 1))  # Add one more column for 'label'

# 각 문장에 대해 반복하면서 DataFrame 생성# 각 문장에서 최대 토큰 수 계산
# max_tokens = max(len(sentence) for sentence in test_data[0])
for i, sentence in enumerate(test_data[0]):
    # DataFrame에 토큰 형태 추가
    for j, token in enumerate(sentence):
        test_df.loc[i, j] = token.form

    # 마지막 열에 'label' 값 추가
    test_df.loc[i, max_tokens] = test_data[1].iloc[i]  # Use .iloc[i] to access the value at index i

# NaN 값을 빈 문자열로 대체
test_df = test_df.fillna('')

# 결과적인 DataFrame 출력
print(test_df) 

        0    1   2    3    4   5    6    7   8    9    ... 639 640 641 642  \
0        은행    에  대출   상담    받  으러    가    아   보    면  ...                   
1        우리  지역구   이   ᆫ데  금태섭   뽑   으면    안   되    지  ...                   
2        ㅋㅋ   복수   하   ᆯ겨                              ...                   
3       당연히   많이   보    ᆯ    수  밖에  ...                ...                   
4        호주   에서   는   남자    들   이   동물   보다  못하    ᆫ  ...                   
...     ...  ...  ..  ...  ...  ..  ...  ...  ..  ...  ...  ..  ..  ..  ..   
21769     '    '  게'    '    '   나    '    '  고동    '  ...                   
21770     '    '  쌍욕   쟁이    '   '    '   무상  연애  주의자  ...                   
21771  #@이름    #   저  도라이   국민   의    힘    에   ᆫ    왜  ...                   
21772     야    는  아직    도  대통령   '    '  병자'   '    '  ...                   
21773    내역    을   잘    보  아야지   ,   그거    ᆫ  가끔    이  ...                   

      643 644 645 646 647 648  
0                           0  

In [11]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
import pandas as pd
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt

okt = Okt()


stop_words = set(['는', '은', '이', '가', '을', '를', '그리고', '그래서', '또한', '그러하니', '그러나', '그리하여', '그리하여금', '그러한', '그러므로', ''])

def process_text(text):
    if isinstance(text, str):
        words = word_tokenize(text)
        filtered_words = [word for word in words if word not in stop_words]
        return ' '.join(filtered_words)
    else:
        return ''

# 각 열에 함수를 적용합니다.
for column_index in train_df.columns:
    train_df[column_index] = train_df[column_index].apply(process_text)

# 처리된 열을 하나의 열로 합칩니다.
train_df['merged_sentences'] = train_df.apply(lambda row: ' '.join(row), axis=1)

print(train_df['merged_sentences'])


0        음 그렇 ᆫ가 ?                                     ...
1        강원도  꼴랑 700 억 주 고 ㅋㅋㅋㅋㅋㅋ 차이나 타운 같은 거나 만들 어 주 네...
2        ' ' 빨갱이 ' ' ' 들 이제 ᆫ 대놓고 축 하하 나 의 과거 도 그렇 고 미래...
3        그런 대사 뭐  지 , ? 소설 같 네                         ...
4        ' ' 미치 ᆫ ' ' ' . 그야말로 사람  기계 부품 취급 . ' ' 빨갱좌빠리...
                               ...                        
84541    키키 같이 입사 도전 ?                                 ...
84542    애초 에 유재석  우리나라 개돼지 국민 서 으 로 나 좋아하 ᆯ 수 있  인간상  ...
84543    대가리 텅 비 ᆫ 년 들 ..                              ...
84544    불 에 타  줄 알 았 음 ... 덥 으면 악몽 꾸 잖아 그렇 게 매일 아침 에 깨...
84545    아니요 ... 아무 도 없 네요                             ...
Name: merged_sentences, Length: 84546, dtype: object


C:\Users\administrator\AppData\Local\Temp\2\ipykernel_8092\1581728357.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df['merged_sentences'] = train_df.apply(lambda row: ' '.join(row), axis=1)


In [13]:
# 각 열에 함수를 적용합니다.
for column_index in test_df.columns:
    test_df[column_index] = test_df[column_index].apply(process_text)

# 처리된 열을 하나의 열로 합칩니다.
test_df['merged_sentences'] = test_df.apply(lambda row: ' '.join(row), axis=1)

print(test_df['merged_sentences'])

0        은행 에 대출 상담 받 으러  아 보 면 직업 의 귀천 바로 알리 어 주 ᆷ    ...
1        우리 지역구  ᆫ데 금태섭 뽑 으면 안 되 지 ? 그래도 자 한 당 뻡아야  겠 지...
2        ㅋㅋ 복수 하 ᆯ겨                                    ...
3        당연히 많이 보 ᆯ 수 밖에 ...                           ...
4        호주 에서  남자 들  동물 보다 못하 ᆫ 존재  라던데 ㅋㅋ            ...
                               ...                        
21769    ' ' 게 ' ' ' 나 ' ' 고동 ' ' ' 이나 대통령  되 겠 다고 하 는데...
21770    ' ' 쌍욕 쟁이 ' ' ' 무상 연애 주의자  ' ' 죄명 ' ' ' 이나 전라도...
21771    # @ 이름 # 저 도라이 국민 의 힘 에 ᆫ 왜 저런 놈 만 있 을까 ? 나이 값...
21772    야  아직 도 대통령 ' ' 병자 ' ' ' 냐 ? 또 철수 없 으 도 대한민국 잘...
21773    내역  잘 보 아야지 , 그거 ᆫ 가끔  고 거의 매일 한 명 당 2-3 만 원 씩...
Name: merged_sentences, Length: 21774, dtype: object


C:\Users\administrator\AppData\Local\Temp\2\ipykernel_8092\606414554.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_df['merged_sentences'] = test_df.apply(lambda row: ' '.join(row), axis=1)


In [21]:
train_label = train_df.iloc[649].values
test_label = test_df.iloc[649].values

In [22]:
test_label

array(['저런', '거', 'ᆯ', '보', '고', '도', '뭐', '', 'ᆫ지', '도', '모르', 'ᆯ', '만큼',
       '관심', '도', '없', '고', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 텍스트를 TF-IDF 벡터로 변환합니다.
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_df['merged_sentences'])
X_test = vectorizer.transform(test_df['merged_sentences'])  # transform만 사용

# 랜덤 포레스트 모델을 생성하고 학습합니다.
rf_model = RandomForestClassifier()
rf_model.fit(X_train, train_label)

# 테스트 데이터에 대한 예측을 수행합니다.
y_pred = rf_model.predict(X_test)

# 정확도를 출력합니다.
accuracy = accuracy_score(test_label, y_pred)
print(f'Accuracy: {accuracy:.2f}')


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def print_scores( test_label, y_pred, pred_proba=None):
    # 필요한 모듈 임포트
    from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, \
        roc_auc_score
    
    # 오차 행렬 계산
    cm = confusion_matrix(test_label, y_pred)
    TN, FP, FN, TP = cm.ravel()
    
    # 다양한 성능 지표 출력
    print('오차행렬 \n', cm)
    print('Accuracy_score(정확도) :', accuracy_score(test_label, y_pred))
    print('Precision(정밀도) : ', precision_score(test_label, y_pred))
    print('Recall(재현율) :', recall_score(test_label, y_pred))
    
    # TNR은 True Negative Rate로, 실제 0인 것 중에서 모델이 정확하게 예측한 비율입니다.
    print('TNR(0을 맞춘 비율) :', TN / (TN + FP))
    
    # F1 score 계산
    print('F1 score :', f1_score(test_label, y_pred))
    
    # 예측 확률이 주어진 경우에만 Roc Auc score 출력
    if pred_proba is not None:
        print('Roc Auc score :', roc_auc_score(test_label, pred_proba))

        # 혼동 행렬 그리기
    cm = confusion_matrix(test_label, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Predicted 0', 'Predicted 1'],
                yticklabels=['Actual 0', 'Actual 1'])
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()


print_scores(test_label,y_pred)

In [ ]:
def sentence_predict(sent):
    sentence=True
    new_data_transformed = vectorizer.transform([sent])
    prediction = rf_model.predict(new_data_transformed)
    
    if prediction == 0:
        sentence = True #정상어

    elif prediction == 1:
        sentence = False #비속어
    return sentence, sent

sentence_predict("씨발")

In [ ]:
%run C:\Users\administrator\Desktop\X_filter\AI_Model\LSTM.ipynb

In [ ]:
def badword_find(sent):
    result = sent
    badword_df = pd.read_excel(r'C:\Users\administrator\Desktop\X_filter\AI_Model\dataset\word_list (완성).xlsx')
    
    found_bad_word = False  # 입력 문장에 단어가 발견되었는지를 나타내는 플래그
    for idx, row in badword_df.iterrows():
        if row["WORD"] in sent:
            # 'WORD'가 입력 문장에 포함된 경우
            new_word = row["대체어"]
            if not pd.isnull(new_word):
                result = result.replace(row["WORD"], new_word)
                found_bad_word = True
            else:
                result = result.replace(row["WORD"], "*" * len(row["WORD"]))
                found_bad_word = True
    
    if not found_bad_word:
        # result = "@" * len(input_sentence)
        result = "혐오 표현입니다."
    return result

In [ ]:
def speak_pre(sent):
    sentence = sentence_predict(sent)
    if sentence[0] == True:
        return sent
    elif sentence[0] == False:
        return badword_find(sent)
    
def speak(sent):
    speak_pre(sent)
    sentence = sentence_predict(sent)
    if sentence[0] == True:
        return sent
    elif sentence[0] == False:
        return badword_find(sent)

In [ ]:
# 특수문자 제거
def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[^가-힣ㄱ-ㅎㅏ|a-zA-Z0-9]','', text)
    return text

In [ ]:
import re
from soynlp.normalizer import repeat_normalize

def clean_and_repeat_normalize(text):
    cleansed_text = cleanse(text)                                             # 특수문자 제거
    normalized_text = repeat_normalize(cleansed_text, num_repeats=2)          # 중복문자 제거
    input_data = re.sub(r'\d', '', normalized_text)                           #  제거
    # normalized_text = spacing(input_data)                                   # 띄어쓰기 보정 

    return input_data

clean_and_repeat_normalize("아버지가방에들어가신다")

In [ ]:
import kss

def final_output():
    input_text = clean_and_repeat_normalize(input())
    sentences = kss.split_sentences(input_text)

    sentences_list = []
    for sentence in sentences:
        speak(sentence)
        sentences_list.append(sentence)

    long_test = ' '.join(sentences_list)
    print(long_test)
    return long_test

In [ ]:
final_output()